<style>
    .center-text{
        text-align: center;
    }
</style>

<div class='center-text'>

# Tarea 1 Aprendizaje Automático

### Julian Ricardo Salazar duarte

#### Universidad Nacional de Colombia

#### 2024-1
</div>


- [1. Preprocesamiento de datos](#s1)
- [2. Análisis exploratorio de datos](#s2)


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

<a id="s1"></a>
### 1. Preprocesamiento de datos

In [2]:
def load_data(filename, skiprows=0, sheet_name=None):
    if sheet_name is None:
        data = pd.read_excel(filename, skiprows=skiprows)
    else:
        data = pd.read_excel(filename, skiprows=skiprows, sheet_name=sheet_name)
    return data

In [3]:
def clean_df(df):

    df = df.dropna(how='all')

    for i in range (len(df.columns)):
        if 'Unnamed' in df.columns[i]:
            df = df.rename(columns={df.columns[i]: df.columns[i-1].split()[0] + ' ' + df.iloc[0, i]})
        elif not pd.isna(df.iloc[0, i]):
            df = df.rename(columns={df.columns[i]: df.columns[i] + ' ' + df.iloc[0, i]})

    df = df.drop(df.index[0])
    df = df.dropna(how='any')
    df.columns = df.columns.str.strip()
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.capitalize()
    df['Fecha'] = df['Fecha'].apply(str)

    df = df.rename(columns={df.columns[2]: 'Marca'})
    
    return df

In [4]:
df = load_data('CC FT 17 Formato de Control de Calidad Café de Trillado.xlsx',
               5, ['CONTROL CALIDAD CAFE TRILLADO J', 'Sheet2'])

df1 = df['CONTROL CALIDAD CAFE TRILLADO J']
df2 = df['Sheet2']

df1 = clean_df(df1)
df2 = clean_df(df2)

In [5]:
df_trillado = pd.concat([df1, df2], axis=0)

for col in df_trillado.columns:
    if isinstance(df_trillado[col].iloc[0], str):
        df_trillado[col] = df_trillado[col].str.lower().str.strip()
    if len(set(df_trillado[col])) == 1:
        df_trillado = df_trillado.drop(columns=[col])

df_trillado['Lote'] = df_trillado['Lote'].str.replace('-', '', regex=False).str.lstrip('0')

df_trillado = df_trillado.reset_index(drop=True)

print(df_trillado.shape)

(106, 9)


##### Label (X)

In [6]:
def replace_date(date):
    date = date.split('-')
    if date[1].isdigit():
        if int (date[1]) < 4:
            return 1
        elif int (date[1]) < 7:
            return 2
        elif int (date[1]) < 10:
            return 3
        else:
            return 4
    else:
        if 'ene' in date[1].lower() or 'feb' in date[1].lower() or 'mar' in date[1].lower():
            return 1
        elif 'abr' in date[1].lower() or 'may' in date[1].lower() or 'jun' in date[1].lower():
            return 2
        elif 'jul' in date[1].lower() or 'ago' in date[1].lower() or 'sep' in date[1].lower():
            return 3
        else:
            return 4

In [7]:
# Fecha toma los valores entre 1 y 4 dividiendo el año en trimestres

df_trillado['Fecha'] = df_trillado['Fecha'].apply(replace_date)

In [8]:
def replace_lote(lote):
    if len(lote) == 7:
        return int(lote[0])
    else:
        return int(lote[:2])

In [9]:
# La columna Lote tiene valores enteros en entre 1 y 23 representando la marca o tipo de cafe

df_trillado['Lote'] = df_trillado['Lote'].apply(replace_lote)

In [10]:
tmp = df_trillado[['Lote', 'Marca']]
tmp = tmp.drop_duplicates(subset='Lote')
print(tmp)

    Lote                       Marca
0      1                 madre laura
1      9                tabi natural
2     10                   don mario
3      7                   don felix
7      8         gesha villabernarda
8      2         madre laura natural
9      5                  doña dolly
12     3    madre laura descafeinado
17    11      monteverde - wush wush
18    12     el ocaso - pink bourbon
28    13   el cedrela - pink bourbon
38    16       gesha villa - natural
39    15         don victor - marago
42     6                  don rafael
43     4                doña rosalba
49    17      familia bedoya castaño
54    18                   don johan
57    19     gesha blue - monteverde
58    20  bourbon sidra - las flores
59    21    don victor - red bourbon
70    22         el ocaso - caturron
72    23             esteban robledo


In [11]:
df = load_data('CC FT 17 Formato de Control de Calidad Café de Trillado.xlsx', sheet_name=['LOTES'])
df = df['LOTES']
df = df[['N°', 'TIPO DE CAFÉ']]
df['TIPO DE CAFÉ'] = df['TIPO DE CAFÉ'].str.lower()
df = df.dropna(how='any')
df = df.set_index('N°')
dict_lote = df.to_dict()['TIPO DE CAFÉ']
dict_lote[23] = tmp[tmp['Lote'] == 23]['Marca'].values[0]

In [12]:
for key, value in dict_lote.items():
    print(f'{key}: {value}')

1: madre laura lavado
2: madre laura natural
3: madre laura descafeinado
4: doña rosalba 
5: doña dolly
6: don rafael pbw 
7: don felix
8: gesha villabernarda
9: don juan tabi
10: doña liceth 
11: monteverde - wush wush
12: el ocaso - pink bourbon
13: el cedrela - pink bourbon 
14: don reinaldo 
15: don victor - maragogipe 
16: gesha natural villabernarda
17: familia bedoya castaño
18: don johan 
19: familia gutierrez - gesha 
20: familia vergara - bourbon sidra 
21: don victor - red bourbon
22: el ocaso - caturron natural 
23: esteban robledo


In [13]:
# se elimina la columna marca ya que es redundante con la columna Lote

df_trillado = df_trillado.drop(columns=['Marca'])

In [16]:
# cantidad, %h y mallas # se deja igual (se normaliozara más adelante)

# Notas de catacion se aplicará One-Hot Encoding

tmp = df_trillado['Notas de catación']
tmp = tmp.str.split(',', expand=True)

In [34]:
def separe_y(row):
    new_row = []
    for r in row:
        tmp = r.split(' y ')
        new_row.extend(tmp)
    return new_row


In [35]:
notes = set()
for i in range(tmp.shape[1]):
    notes = notes.union(set(tmp[i].str.strip()))

notes = list(notes)
notes = [note for note in notes if note is not None]
notes = separe_y(notes)
notes.sort()

In [25]:
print(tmp.shape[1])

6


In [ ]:
notes

In [29]:
print(notes)

{'residual a cascara de mandarina', 'acidez media citrica como naranja', 'acidez citrica como la naranja', 'cuerpo aterciopelado', 'limonciollo', 'nueces', 'final prolongado', 'melao de panela', 'cuerpo cremoso medio y final frutal prolongado.', 'residual frutal a naranja', 'acidez jugosa como tamarindo', None, 'acidez brillante y residual frutal a naranja', 'manzana roja', 'frutas maduras', 'uva', 'rosas', 'naranja', 'final prolongado a un toque frutal', 'te de cedron', 'jalea', 'almendras', 'fresas', 'cuerpo cremoso', 'caramleo', 'cuerpo sedoso y final limpio y prolongado', 'panela', 'final frutal', 'cacao', 'acidez jugosa citrica', 'clavos de olor', 'caña de azucar', 'azucar morena', 'frutas amarillas', 'cuerpo cremoso y final dulce como la melaza', 'cuerpo sedoso y final limpio', 'acidez malica brillante', 'limon', 'fresa', 'fnal dulce y prolongado', 'ciruela madura', 'acidez malica', 'chocolate negro', 'jazmin', 'maracuya', 'vainilla', 'miel', 'cuerpo cremoso y final frutal a dura

In [14]:
for col in df_trillado.columns:
    print(f'{col}: {len(set(df_trillado[col]))}')

Fecha: 4
Lote: 22
Cantidad: 66
%h %: 13
Mallas #: 3
Notas de catación: 28
Puntaje n°: 10
Responsable: 2


<a id="s2"></a>
### 2. Análisis exploratorio de datos